In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymongo
from pprint import pprint
import joblib

In [2]:
ratings_matrix = pd.read_pickle("Data/test_matrix.pkl")

ratings_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175993,175995,175999,176001,176003,176007,176075,176157,176211,176271
userId,,,,,,,,,,,,,,,,,,,,,
9,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#load movies

movies = pd.read_csv('Data/movies_sml.csv')

In [4]:
#load ratings

ratings = pd.read_csv('Data/ratings_sml.csv')

In [5]:
R = ratings_matrix.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [6]:
# #load other vectors needed to compute predictions

# U = joblib.load('Data/U.pkl')
# Vt = joblib.load('Data/Vt.pkl')
# sigma = joblib.load('Data/sigma.pkl')

In [7]:
# # calculate the dot product of our vectors
# all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [8]:
all_user_predicted_ratings = joblib.load('Data/all_user_predicted_ratings')

In [9]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = ratings_matrix.columns)
preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175993,175995,175999,176001,176003,176007,176075,176157,176211,176271
0,4.043145,-0.031401,-0.109841,-0.040066,-0.007383,-0.362918,0.138744,-0.059072,0.099973,0.204405,...,-0.002675,-0.002703,-0.002703,-0.002703,-0.002732,-0.002617,-0.002744,-0.003168,-0.003729,-0.003049
1,2.967341,0.112618,-0.650235,0.084441,-0.542322,0.190666,-0.334271,-0.013707,-0.149256,0.302472,...,0.001898,0.001875,0.001875,0.001875,0.001851,0.001945,0.002245,0.001783,0.003136,0.002110
2,0.355257,-0.414077,0.207500,-0.050163,0.178221,2.574964,0.237428,-0.055524,0.136858,0.469555,...,-0.000418,-0.000219,-0.000219,-0.000219,-0.000020,-0.000816,-0.000617,-0.000151,-0.001692,-0.000792
3,-0.215976,-0.141312,-0.218790,0.037475,-0.053217,0.369246,-0.140024,-0.031138,-0.099683,-0.607257,...,0.006418,0.006385,0.006385,0.006385,0.006352,0.006485,0.006283,0.007180,0.004884,0.005402
4,1.542347,0.393789,-0.016029,0.048731,-0.132489,0.158159,-0.108102,0.055406,-0.064908,0.346607,...,0.000917,0.001000,0.001000,0.001000,0.001083,0.000750,0.001514,0.000832,0.001296,0.000532


In [10]:
def recommend_movies(predictions, userId, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userId - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.userId == (userId)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

#     print (f"User {0} has already rated {1} movies. format(userID, user_full.shape[0]")
#     print 'Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations)
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    rec_movies = recommendations['Title']
    already_rated = user_full['Title_x']
    return  already_rated, rec_movies

In [13]:
already_rated, predictions = recommend_movies(preds, 60, movies, ratings, 20)

In [14]:
predictions

423                       Schindler's List
215     Star Wars: Episode IV - A New Hope
293                           Forrest Gump
102                             Braveheart
6587                             Inception
6003                                    Up
5575                                WALL·E
1420                     Life Is Beautiful
1950                             Gladiator
8495                          Interstellar
249                 Léon: The Professional
485                                  Fargo
3110                          Finding Nemo
7820                      Django Unchained
8928                    The Imitation Game
5957                  Inglourious Basterds
5748                   Slumdog Millionaire
5059                           Ratatouille
2950                   Catch Me If You Can
9237                            Inside Out
Name: Title, dtype: object